### 글로벌 도시 현재시간 & 날씨 crawling

In [1]:
import time
import csv
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# driver 설정
from selenium import webdriver 
from selenium.webdriver.common.by import By
        
# chrome driver
driver = webdriver.Chrome()

In [3]:
# url='https://www.accuweather.com/'

# driver.get(url)
# time.sleep(1.5)
# browser.find_element(By.CLASS_NAME, 'search_input').click()
# browser.find_element(By.CLASS_NAME, 'search_input').send_keys('날씨')
# browser.find_element(By.CLASS_NAME, 'btn_search').click()

In [ ]:
# # 체감, 습도, 풍향
# data = browser.find_element(By.CLASS_NAME, 'summary_list').text
# data.split(" ")